list google bucket url map?


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials = path.join(PROFILE_PATH, '.credentials/spahaha-ea443-a78ab2269985.json');
var https = require('https');
var util = require('util');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

function listUrlMaps(project) {
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps`
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = {
                    hostRules: cur.hostRules,
                    pathMatchers: cur.pathMatchers
                };
                return obj;
            }, {})
        });
}

function listTargetProxies(project, urlMap) {
    var params = {};
    if(urlMap) {
        params = {
            filter: `urlMap="https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps/${urlMap}"`
        }
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies`,
            params
        }))
        .then(res => (res.data.items || []).map(p => p.name));
}

function listGlobalForwards(project, proxy) {
    var params = {};
    if(proxy) {
        params = {
            filter: `target="https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies/${proxy}"`
        }
    }
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/forwardingRules`,
            params
        }))
        .then(res => (res.data.items || []).map(p => p.name));
}

function listBackendBuckets(project, ) {
    return authorizeGoogle(credentials)
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets`
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = cur.bucketName;
                return obj;
            }, {})
        });
}

module.exports = {
    listUrlMaps,
    listTargetProxies,
    listGlobalForwards,
    listBackendBuckets
};


add a google bucket web map?


In [ ]:

function createBackendBucket(project) {
    //https://www.googleapis.com/compute/v1/projects/{project}/global/backendBuckets


}

module.exports = {
    createBackendBucket
};


test bucket web map?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    createBackendBucket,
} = importer.import('add google bucket web map');
var project = 'spahaha-ea443';
var domain = 'sheet-to-web.sheet-to-web.com';

describe('adding a subdomain to a load balancer', () => {
    
    it('should add a bucket backend', () => {
        return createBackendBucket(project, domain)
            .then(items => {
                assert(false, 'should have added a backend');
            })
    })
    
    
})


how to authorize google services?

In [ ]:
var {auth} = require('google-auth-library');
var jwt_client, google_auth;
var headers = {};
var util = require('util');
var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/compute',
    'https://www.googleapis.com/auth/cloud-platform'
];

function authorizeGoogle(creds) {
    return auth.getClient({
        keyFile: creds,
        scopes: GOOGLE_AUTH_SCOPE
    });
}

module.exports = authorizeGoogle;


test list url map?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    listUrlMaps,
    listTargetProxies,
    listGlobalForwards,
    listBackendBuckets
} = importer.import('list google bucket url map');
var project = 'spahaha-ea443';
var map = 'web-map';
var proxy = 'web-map-target-proxy-5';

describe('listing subdomains from load balancer', () => {
    
    it('should list buckets maps', () => {
        return listUrlMaps(project)
            .then(items => {
                console.log(items[Object.keys(items)[0]].hostRules);
                console.log(items[Object.keys(items)[0]].pathMatchers);
                console.log(items[Object.keys(items)[0]].pathMatchers[0].pathRules);
                assert(Object.keys(items).length > 0, 'should have a url mapper');
                
                return listTargetProxies(
                    project,
                    map);
            })
            .then(proxies => {
                console.log(proxies);
                assert(proxies.length > 0, 'should have a proxy');
            
                return listGlobalForwards(
                    project,
                    proxy);
            })
            .then(rules => {
                console.log(rules);
                assert(rules.length > 0, 'should have a rule');
            
                return listBackendBuckets(project);
            })
            .then(buckets => {
                console.log(buckets);
                assert(Object.keys(buckets).length > 0, 'should have a bucket');
            });
    })
    
    
})
